<a href="https://colab.research.google.com/github/dakyommii/study/blob/main/gan_atoi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [52]:
# import random
import os
from os import listdir
from os.path import isfile, join
import PIL
from PIL import Image
from glob import glob

import IPython.display as display

import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [53]:
DATA_PATH = "/content/drive/My Drive/atoi_gan_fin" 

In [65]:
MODEL_SAVE_PATH = os.path.join("/content/drive/My Drive/results")

In [55]:
np.random.seed(3)

In [56]:
#from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255)
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   rotation_range = 0.2,
                                   brightness_range = (0.2, 0.8),
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True,
                                   vertical_flip = True,
                                   fill_mode='nearest'
                                   )

# test_datagen = ImageDataGenerator(rescale = 1./255)
# training_set = train_datagen.flow_from_directory('/content/drive/My Drive/gan_atoi_fin',
#                                                  target_size = (64,64),
#                                                  batch_size = 32,
#                                                  class_mode = 'binary')

# classifier.fit_generator(training_set, steps_per_epoch=800,
#                          epochs=25,
#                          validation_steps = 125)

In [70]:
file_name_in_dir = []

for root,dirs,files in os.walk(DATA_PATH):
  for fname in files:
    full_fname=os.path.join(root,fname)
    file_name_in_dir.append(full_fname)

for file_image in file_name_in_dir:
      #print(file_image)
  img = load_img(file_image)
  x=img_to_array(img)
  x=x.reshape((1,)+x.shape)

  i=0
  for batch in train_datagen.flow(x,save_to_dir=MODEL_SAVE_PATH,save_prefix='characters', save_format='png'):
    i+=1
    if i>5:
      break

In [38]:
# np.random.seed(1)
# tf.random.set_seed(1)

# learning_rate = 0.001
# dropout_rate=0.5
# epochs=50
# batch=20

# def create_model():
#   model=keras.Sequential()

#   model.add(layers.Conv2D(64,kernel_size=3,padding="SAME",activation='relu', input_shape=(150,150,3)))
#   model.add(layers.MaxPool2D(padding='SAME'))

#   model.add(layers.Conv2D(64,kernel_size=3,padding="SAME",activation='relu'))
#   model.add(layers.MaxPool2D(padding='SAME'))

#   model.add(layers.Conv2D(128,kernel_size=3,padding="SAME",activation='relu'))
#   model.add(layers.MaxPool2D(padding='SAME'))

#   model.add(layers.Flatten())
#   model.add(layers.Dropout(dropout_rate))
#   model.add(layers.Dense(512,activation='relu'))

#   model.add(layers.Dense(1,activation='sigmoid'))

#   return model

In [ ]:
# batch_size = 32
# img_height = 256
# img_width = 256

In [71]:
# img_path = list(glob(str(DATA_PATH + '/*.png')))

# len(img_path)

# (x_train, y_train), (x_test,y_test) = img_path[:10], img_path[10:]

# # for image_path in img_path[: ]:
# #   display.display(Image.open(str(image_path)))